# Subreddit Growth: How many unique subreddits were there at the beginning of 2018?

In [3]:
# Reading data from hdfs cluster on orion11
# In here, I am using reservoir sample because the spark running out of memory and unable to read the full data set
df_2018_jan = spark.read.json('hdfs://orion11:23001/RES-RC_2018-01.zst')

In [4]:
# Get schema
df_2018_jan.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- subreddit_type: string (nullable = true)



In [5]:
df_2018_jan.createOrReplaceTempView("df_view_2018_jan")

# Subreddit Growth: How many unique subreddits were there at the beginning of 2018?
spark.sql("SELECT DISTINCT subreddit AS unique_subreddits FROM df_view_2018_jan").count()

50974

In [6]:
df_2018_dec = spark.read.json('hdfs://orion11:23001/RES-RC_2018-12.zst')

In [7]:
df_2018_dec.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_created_utc: long (nullable = true)
 |-- author_flair_background_color: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_richtext: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- a: string (nullable = true)
 |    |    |-- e: string (nullable = true)
 |    |    |-- t: string (nullable = true)
 |    |    |-- u: string (nullable = true)
 |-- author_flair_template_id: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_flair_text_color: string (nullable = true)
 |-- author_flair_type: string (nullable = true)
 |-- author_fullname: string (nullable = true)
 |-- author_patreon_flair: boolean (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- can_mod_post: boolean (nullable = true)
 |-

In [8]:
df_2018_dec.createOrReplaceTempView("df_view_2018_dec")

# Subreddit Growth: How many unique subreddits were there at the end of 2018?
spark.sql("SELECT DISTINCT(subreddit) AS unique_subreddits FROM df_view_2018_dec").count()

56146

In [9]:
# Print output answer
print("The number of unique subreddits at the beginning of 2018: {}".format(Out[5]))
print("The number of unique subreddits at the end of 2018: {}".format(Out[8]))

The number of unique subreddits at the beginning of 2018: 50974
The number of unique subreddits at the end of 2018: 56146


Using the reservoir sample, we find out that there are about 50974 unique subreddit at the beginning of 2018 and 56146 unique subreddit at the end of 2018. This number might not be accurate as I am using reservoir sample of the data, but it should be close enough.

# User Growth: How many active users does Reddit have now compared to the past?

In [3]:
# Reading data from hdfs cluster on orion11
# Reading the very first data reddit has vs the last data from reddit
df_2005_jan = spark.read.json('hdfs://orion11:23001/RC_2005-12.bz2')
df_2020_dec = spark.read.json('hdfs://orion11:23001/RES-RC_2020-12.zst')

In [10]:
# Get 2005 schema
df_2005_jan.printSchema()

root
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: long (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: boolean (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [ ]:
# Get 2020 schema
# df_2020_dec.printSchema()
# The scheema is too long so I will just comment it

In [18]:
# January 2005
# So my assumption for active user is a user that post something (comment/subreddit) within the month that we are analyzing.
df_2005_jan.createOrReplaceTempView("df_view_2005_jan")

# Subreddit Growth: How many unique subreddits were there at the beginning of 2018?
active_user_2005_jan = spark.sql("SELECT DISTINCT author AS active_user FROM df_view_2005_jan").count()


In [4]:
# December 2020
# So my assumption for active user is a user that post something (comment/subreddit) within the month that we are analyzing.
df_2020_dec.createOrReplaceTempView("df_view_2020_dec")

# Subreddit Growth: How many unique subreddits were there at the beginning of 2018?
active_user_2020_dec = spark.sql("SELECT DISTINCT author AS active_user FROM df_view_2020_dec").count()


In [23]:
# Print output answer
print("The number of active users at jan 2005: {}".format(active_user_2005_jan))
print("The number of active users at dec 2020: {}".format(active_user_2020_dec))
print("Growth percentage: {}".format((active_user_2020_dec - active_user_2005_jan)/active_user_2020_dec * 100))

The number of active users at jan 2005: 394
The number of active users at dec 2020: 2906214
Growth percentage: 99.98644284281887


# Best Comment Award: Choose a particular day and determine what the most upvoted comment was. (Include the comment in your report, of course!)

In [48]:
# Finding the the maximum score on 12-1-2020
spark.sql("SELECT MAX(score) as max_score FROM (SELECT * FROM df_view_2020_dec WHERE from_unixtime(created_utc, 'MM dd yyyy') == '12 01 2020') AS t").show()


+---------+
|max_score|
+---------+
|    26280|
+---------+



In [59]:
# Get the most upvoted comment and author
most_upvoted_comment = spark.sql("SELECT author, body as most_upvoted_comment FROM df_view_2020_dec WHERE score == 26280 AND from_unixtime(created_utc, 'MM dd yyyy') == '12 01 2020'")
most_upvoted_comment.cache()
most_upvoted_comment.show()

+--------+--------------------+
|  author|most_upvoted_comment|
+--------+--------------------+
|ThatKiwi|I've been working...|
+--------+--------------------+



In [62]:
# Printing the whole comment
author_comment = most_upvoted_comment.collect()
print("author: ", author_comment[0][0])
print("comment: ", author_comment[0][1])

author:  ThatKiwi
comment:  I've been working from home and as my employer, like many others, are soul sucking demons not fit for this earth, they track you in just about every way possible because it's simply inconceivable to think people would just do the work they're paid to do, regardless of the setting they're in.

If I dont interact with the computer for more than five minutes it registers me as away, heavens forbid I have to poop or something. 

I got spoken to the very first week of WFH and told "you have to be available during work hours" by a supervisor who is seemingly never available but was blind to the irony I guess.

In any event, I found that if you navigate to a certain portion of our 1970s software you can endlessly type. I promptly went outside and found a rock which now holds down my space bar for me anytime I need to walk away for a minute. 

That rock is, in my eyes, now one of the most important objects in my world and it was free.

Edit: Fixed typos. I feel sham

For this question, I use a reservoir sample for December 2022 data set. The particular date that I chose is December 1st, 2020. To answer this question, I filtered out all the comments that are not created on December 1st, 2020. Then, to get the most upvote, I simply use max funtion to the score column. Lastly, get the comments on that day which has n number of upvotes. The result should show all the comments that receive that number of upvotes.

The most popular comment on December 1st, 2020 is a comment/post posted by ThatKiwi. I use reservoir sample for this question too, which makes this comment might not be the most popular on that day. But this comment should be one of the most popular one on that day.

# Top Comments: For the user you found in the previous question, find their five most-upvoted comments overall across the entire dataset. Do they post highly-upvoted comments often, or are they a “one hit wonder?”

In [1]:
is_done = False # A boolean to exit the loop
counter = 0
most_popular_posts = {} # Map to store the comment and number of upvotes

num_of_years = 0 # A counter of how many years we have done so far
num_of_months = 0 # A counter of how many months we have done so far

# Since we have a really big data, it takes a long time to process all the years from 2004-2020, therefore
# these two variables below is to limit how many years and months do you want to scan
num_of_years_target = 1 
num_of_months_target = 6

# Iterate through the number of years we have which is 2004 - 2020
for i in range(20, 4, -1):
    if is_done or num_of_years == num_of_years_target:
        break
    
    # Iterate through the number of months 1 - 12
    for j in range(12, 0, -1):
        if is_done or num_of_months == num_of_months_target:
            break
            
        file_dir = ''
        if i == 5:
            file_dir = 'hdfs://orion11:23001/RC_2005-12.bz2'
        elif i > 5 and i < 18:
            file_dir = 'hdfs://orion11:23001/RC_20{:02d}-{:02d}.bz2'.format(i, j)
        elif i >= 18:
            file_dir = 'hdfs://orion11:23001/RES-RC_20{:02d}-{:02d}.zst'.format(i, j)
    
        # Read a file from hdfs
        df = spark.read.json(file_dir)
        df.createOrReplaceTempView("df_view")
        
        # Get the number of comments/posts 'ThatKiwi' user has
        total_post = spark.sql("SELECT count(*) AS ct FROM df_view WHERE author == 'ThatKiwi'")
    
        print("Number of posts for month {}: {}".format(str(j), str(total_post.take(1)[0]['ct'])))
        if (total_post.take(1)[0]['ct'] < 10):
            counter += 1
            if counter == 7:
                is_done: True

        # Get the most upvoted comment and store it to the map
        most_popular_post = spark.sql("SELECT body, score FROM df_view WHERE author == 'ThatKiwi'  ORDER BY score DESC LIMIT 1")
        if (most_popular_post.count() > 0):
            post = most_popular_post.take(1)
            most_popular_posts[post[0]['body']] = post[0]['score']
        
        num_of_months += 1
    
    num_of_years += 1
if len(most_popular_posts) == 0:
    print("The user do not have any other popular posts")
else:
    # Print list of comments and upvotes
    for key, value in most_popular_posts.items():
        print("post: " + key + " | upvotes: " + str(value))
    

Number of posts for month 12: 1
Number of posts for month 11: 0
Number of posts for month 10: 0
Number of posts for month 9: 1
Number of posts for month 8: 1
Number of posts for month 7: 0
post: I've been working from home and as my employer, like many others, are soul sucking demons not fit for this earth, they track you in just about every way possible because it's simply inconceivable to think people would just do the work they're paid to do, regardless of the setting they're in.

If I dont interact with the computer for more than five minutes it registers me as away, heavens forbid I have to poop or something. 

I got spoken to the very first week of WFH and told "you have to be available during work hours" by a supervisor who is seemingly never available but was blind to the irony I guess.

In any event, I found that if you navigate to a certain portion of our 1970s software you can endlessly type. I promptly went outside and found a rock which now holds down my space bar for me a

Apparently the user did not post any other comment on December 2020. Additionally, as you can see above from the previous months data, it seems like she/he only occasionally. We can see that within the last 6 months, he/she only posted 3 times. Out of those three comments, only one comments get a lot of upvotes. The rest of the comments only get 1 and 3 upvotes as you can see above. Therefore, it is safe to assume that her/his comment is a "one hit wonder".

# Ban Hammer: Based on user activity, determine which subreddits have been recently banned.

In [6]:
# Get 2020 november data
df_2020_nov = spark.read.json('hdfs://orion11:23001/RES-RC_2020-11.zst')
df_2020_nov.createOrReplaceTempView("df_view_2020_nov")

In [7]:
# Get all subreddits that exist in november 2020 but not exist in december 2020
# This approach does not give guarantee that the subreddit is banned because we are using reservoir sample
recent_banned_subreddit = spark.sql("SELECT DISTINCT subreddit FROM (SELECT subreddit FROM (SELECT subreddit, COUNT(*) AS num_of_comments FROM df_view_2020_nov GROUP BY subreddit) AS t1 WHERE num_of_comments > 100 AND subreddit NOT IN (SELECT DISTINCT subreddit FROM df_view_2020_dec)) AS t3")

recent_banned_subreddit.cache()
recent_banned_subreddit.show()

+--------------------+
|           subreddit|
+--------------------+
|         csci040temp|
|          DoomerGvng|
|            Vindicta|
|dadswhodidnotwant...|
|ChiefsvsRavensLiveTv|
| RadicalChristianity|
|     YoungPrettyHoes|
|        megnuttleaks|
|        donkybooties|
|              hftyty|
|               ismos|
|FitnessGuidesSharing|
|        CallMeCarson|
+--------------------+



For this last question, my approach is to compare the subreddit from the month before the current month. The data that I am using for this project is RES-RC_2020-12.zst and RES-RC_2020-11.zst. First, I filtered the subreddit that has at least 100 comments on it. Next, I am comparing the subreddits in November and in December 2020. If the subreddits in November have at least 100 comments and 0 comments in December, it means the subreddit is banned.